## 4.2 Gram-Schmidt Orthogonalization

In [ ]:
import numpy as np
from scripts.lu import backward


We implement the classic Gram-Schmidt orthogonalization method.

In [ ]:
def gram_schmidt(A):
    n = A.shape[0]
    Q = np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(A[:,i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
     
    return Q

*Additional code details*
- Rather than storing $\tilde{q}_i$ as individual vectors, we collect it in the i-th column of the matrix $Q$.
- The `numpy` function `inner` gives an efficient implementation of the scalar product without python loops and access of individual entries. 
- The `numpy` function `linalg.norm` gives an efficient implementation on the Euclidean norm. With the additional keyword argument `ord`, this function can also compute other vector and matrix norms.

#### Example 4.7 (Gram-Schmidt)
We apply our implementation to the 3x3 Hilbert-Matrix
$$
A = 
\begin{pmatrix} 1 & 1/2 & 1/3\\ 1/2 & 1/3 & 1/4\\1/3 & 1/4 & 1/5\end{pmatrix}.
$$
To emphasize the effect of finite precision artithmetic, we use `half` precision floating point numbers.

In [ ]:
A = np.array([[1,     1 / 2, 1 / 3],
              [1 / 2, 1 / 3, 1 / 4],
              [1 / 3, 1 / 4, 1 / 5]], dtype=np.half)
Q = gram_schmidt(A)
print(Q)

Now we check to what extent the result is really an orthogonal matrix. To multiply two two-dimensional `numpy` arrays (i.e., matrices), we have to use the `@` operator. The `*` operator computes the component-wise product of two arrays.

In [ ]:
print(Q @ Q.transpose())
err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'||Q * Q^T - I||_2 = {err_ort}')

#### Example 4.8 (QR factorization using Gram-Schmidt orthogonalization)

We modify the above implementation to compute the QR factorisation of a matrix.

In [ ]:
def qr_gram_schmidt(A):
    n = A.shape[0]
    Q, R = np.zeros_like(A), np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(A[:,i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
        for j in range(i, n):
            R[i, j] = np.inner(Q[:, i], A[:, j])
    return Q, R

We test this using the linear system $Ax=b$ with
$$
A = 
\begin{pmatrix} 1 & 1 & 1\\ 0.01 & 0 & 0.01\\ 0 & 0.01 & 0.01\end{pmatrix}
\qquad
b =
\begin{pmatrix} 1\\ 0\\ 0.02\end{pmatrix},
$$
which has the exact solution
$$
x =
\begin{pmatrix} -1\\ 1\\ 1\end{pmatrix}
$$
To emphasize the effects form floating-point arithmetic, we again use `half` precision floating point numbers.

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

Q, R = qr_gram_schmidt(A)

b2 = np.dot(Q.transpose(), b)
x = backward(R, b2)

print('x =', x)
print('x_ex = ', x_ex)

The resulting relative error is

In [ ]:
rel_err = np.linalg.norm(x - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x_ex|| = {rel_err}')

In the Frobenius norm, the error in the orthogonality of $Q$ is

In [ ]:
err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'||Q * Q^T - I||_2 = {err_ort}')

#### Example 4.10 (Modified Gram-Schmidt Method)

As the Gram-Schmidt orthogonalization method is not stable, we implement the modified Gram-Schmidt method.

In [ ]:
def gram_schmidt_mod(A):
    n = A.shape[0]
    Q = np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(Q[:, i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q 

Applied to the 3x3 Hilbert matrix, yields with `half` floating point representation

In [ ]:
A = np.array([[1,     1 / 2, 1 / 3],
              [1 / 2, 1 / 3, 1 / 4],
              [1 / 3, 1 / 4, 1 / 5]], dtype=np.half)

Q = gram_schmidt_mod(A)
err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'Q =\n{Q}')
print(f'||Q * Q^T - I||_2 = {err_ort}')

The orthogonality of the matrix $Q$ has improved by a factor of about 3.7. We can now also use this method to implement an improved QR factorization.

In [ ]:
def qr_gram_schmidt_mod(A):
    n = A.shape[0]
    Q, R = np.zeros_like(A), np.zeros_like(A)
    
    for i in range(n):
        Q[:, i] = A[:, i]
        for j in range(i):
            Q[:, i] -= np.inner(Q[:, i], Q[:, j]) * Q[:, j]
        Q[:, i] /= np.linalg.norm(Q[:, i])
        for j in range(i, n):
            R[i, j] = np.inner(Q[:, i], A[:, j])
    return Q, R

We apply this to solve the same linear system as above and obtain

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

Q, R = qr_gram_schmidt_mod(A)
b3 = np.dot(Q.transpose(), b)
x2 = backward(R, b3)
print(f'x = {x2}')

The solution is clearly still not very good. Quantifying this shows

In [ ]:
rel_err = np.linalg.norm(x2 - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x_ex|| = {rel_err}')

err_ort = np.linalg.norm(Q @ Q.transpose() - np.identity(Q.shape[0]), ord=2)
print(f'||Q * Q^T - I||_2 = {err_ort}')

Even though we have improved the orthogonality of the matrix $Q$ by a factor of 70, the relative error has improved by less than a factor of 2.